# Word Analogy and Word Debiasing Using Word Embeddings

The purpose of this project is to create a word analogy with the help of pre-trained word embeddings. By the end of this project, we can create an algorithm such that if we input large -> larger, then the computer can perceive small -> smaller.

In addition to that, word debiasing algorithm will also be implemented because some words should always be in a neutral space in a finite word-embedding dimensions, i.e should not be biased towards positive or negative section in word-embedding dimensions.

We only need numpy library for this project, so let's import numpy library first.

In [13]:
import numpy as np

Next, let's define a function to read the pre-trained word embeddings. The pre-trained model of word embeddings used in this project can be seen at https://nlp.stanford.edu/projects/glove/. This global vectors of words represents 100-dimension of embedding space.

In [14]:
def read_glove_vecs(glove_file):
    
    with open(glove_file,'r', encoding="utf8", ) as f:
        
        words = set()
        word_to_vec_map = {}
        
        print(f)
        for line in f:
            
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [15]:
words, word_to_vec_map = read_glove_vecs('glove.6B.100d.txt')

<_io.TextIOWrapper name='glove.6B.100d.txt' mode='r' encoding='utf8'>


In the code above, `words` parameter represents set of words contained in the global vectors, while `word-to_vec_map` represents the dictionary mapping of set of words to their global vectors representation.

After reading the pre-trained global vectors, now we can start to build word analogy algorithm. As we know already, in order to create a word analogy, we need to compute the distance between words in high-dimensional embedding space. The distance between words can be a good measure how similar or disimilar each word is to the others.

In order to compute the distance, cosine similarity algorithm will be used. Below is the mathematical equation on how to investigate the similarity using cosine similarity:

$$\text{CosSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta)$$

$$ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$$

In above equation, $u.v$ is the dot products between word vectors $u$ and $v$, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. 

If the $CosSimilarity(u,v)$ is close to 1, it means that the two words are similar, while if they are dissimilar, then the value will be lower.

Let's define a function to compute cosine similarity based on the equation above.

In [6]:
def cosine_similarity (vector_u, vector_v):
    
    dot_product = np.dot(vector_u, vector_v)
    
    norm_u = np.sqrt(np.sum(vector_u**2))
    norm_v = np.sqrt(np.sum(vector_v**2))
    
    cosine_similarity = dot_product/(norm_u*norm_v)
    
    return cosine_similarity

To get better intuition about what the result of this cosine similarity represents, let's take a look at the image below.

<img src="image_vec.png" width="700" height="200">

From image above, similar word vectors can be achieved for example if we have words "german" and "spanish" because both words represent nationality of a person. If we have words for example "tiger" and "stadium" then probably we will get a dissimilar vector. However, if we have words such as "Indonesia-Jakarta" and "Berlin-Germany", then we get a similar vectors but they are pointing to the opposite directions.  

Let's prove this theory with some validations.

In [7]:
german = word_to_vec_map["german"]
spanish = word_to_vec_map["spanish"]
tiger = word_to_vec_map["tiger"]
stadium = word_to_vec_map["stadium"]
indonesia = word_to_vec_map["indonesia"]
jakarta = word_to_vec_map["jakarta"]
berlin = word_to_vec_map["berlin"]
germany = word_to_vec_map["germany"]

print("cosine_similarity(german, spanish) = ", cosine_similarity(german, spanish))
print("cosine_similarity(tiger, stadium) = ",cosine_similarity(tiger, stadium))
print("cosine_similarity(indonesia - jakarta, berlin - germany) = ",cosine_similarity(indonesia - jakarta, berlin - germany))

cosine_similarity(german, spanish) =  0.60572438482194
cosine_similarity(tiger, stadium) =  0.24485984266758953
cosine_similarity(indonesia - jakarta, berlin - germany) =  -0.6505975768945929


As we can see from the result above, the word "german" and "spanish" has the highest cosine similarity between them because both of them represent the nationality of a person. Meanwhile the word "tiger" and "stadium" have a low cosine similarity between them because they are not correlated by any chance. Finally, the word "Indonesia - Jakarta" and "Berlin - Germany" are similar with each other but the vectors are pointing in the opposite directions since one of the word represent a country and its capital, while the other represent a country's capital and its country name.

With this cosine algorithm, we can finaly create our own word analogy.

## Word Analogy

With cosine similarity algorithm, now we can build our own word analogy. With word analogy, the computer can find out about the related words with respect to our input words. Let's say we give the computer an example that "small" -> "smaller". If we input a word "large", then the computer will hopefully predict that the appropriate word output should be "larger". Let's define a function to create word analogy.

In [11]:
def word_analogy(word_1, word_2, word_3, word_to_vec_map):
    
    word_1, word_2, word_3 = word_1.lower(), word_2.lower(), word_3.lower()
    
    vector_a, vector_b, vector_c = [word_to_vec_map.get(x) for x in [word_1, word_2, word_3]]
    
    bag_of_words = word_to_vec_map.keys()
    max_cos_similarity = -1000
    analogous_word = None
    
    input_words = set([word_1, word_2, word_3])
    
    for i in bag_of_words:
        
        if i in input_words:
            continue
        
        cos_similarity = cosine_similarity((vector_b-vector_a), (word_to_vec_map[i]-vector_c))
        
        if cos_similarity > max_cos_similarity:
            max_cos_similarity = cos_similarity
            analogous_word = i
    
    return analogous_word

Next, let's do the fun part. After applying the function above, we can now give a logical pair of words to the computer, and then let the computer guess the logical word of our input word. Let's say we give an example of paired logical words "italy" -> "italian" to the computer. Then if we give a new example such as "france", hopefully the computer will predict that the output is "french" based on the example that we gave to the computer.

In [12]:
trials_to_try = [('italy', 'italian', 'france'), ('paris', 'french', 'jakarta'), ('man', 'woman', 'boy'), ('small', 'smaller', 'big')]
for triad in trials_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, word_analogy(*triad,word_to_vec_map)))

italy -> italian :: france -> french
paris -> french :: jakarta -> indonesian
man -> woman :: boy -> girl
small -> smaller :: big -> bigger


The result looks pretty good! From several examples above, it can be seen that the computer predicted the output words correctly in all of the paired logical words that we gave as examples. From the result above, we can see that cosine similarity is a good algorithm in order to find similarities of word vectors in a high dimensional embedding space.

Next, let's talk about word debiasing!

## Word Debiasing

Because all of the words have been mapped into their corresponding embedding in high dimensional spaces, there is no doubt that there might be a potential of high bias among words. While this bias is not a problem for certain words, but for certain words this might be problematic. Let's see what this means in this word debiasing section.

Suppose we want to empashize the word embedding that encodes the concept of gender, like man or woman, boy or girl, and father or mother. We then measure the similarity between these words and as the result, we get the vetor representation of gender.

In [22]:
vec_gender_1 = word_to_vec_map['woman'] - word_to_vec_map['man']
vec_gender_2 = word_to_vec_map['mother'] - word_to_vec_map['father']
vec_gender_3 = word_to_vec_map['girl'] - word_to_vec_map['boy']

vec_gender = (vec_gender_1 + vec_gender_2 + vec_gender_3)/3

In [23]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['cristiano', 'marie', 'sophie', 'lionel', 'liana', 'frank', 'danielle', 'ruben', 'katy', 'gwenn']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], vec_gender))
    

List of names and their similarities with constructed vector:
cristiano -0.26978141682617657
marie 0.27827313109500895
sophie 0.29482186776178243
lionel -0.1978422637542166
liana 0.11854306625152639
frank -0.2666248280507678
danielle 0.2157615895497447
ruben -0.1846352672830835
katy 0.24279711205080473
gwenn 0.06087660665040621


As we can see from the output above, we get either a value below 0 (negative value) or above 0 (positive value). The positive value favors slightly towards the name which has an association to female names (marie, sophie, liana, danielle, katy, gwenn) while the negative value favors slightly towards the name which has an association to male names (cristiano, lionel, frank, ruben). In the case of people's name, it is not surprising that the vector representation of different names has a bias towards people's gender.

However, let's take a look at different kind of examples.

In [24]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'make-up', 'receptionist', 
             'technology',  'fashion', 'babysitter', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], vec_gender))

Other words and their similarities:
lipstick 0.26239475958178377
guns -0.026486816826763952
science -0.02950717932770746
arts -0.009740465149808631
literature 0.0335872135277116
warrior -0.0924716650655859
doctor 0.04784108101903545
make-up 0.19464814971231043
receptionist 0.2863207848022488
technology -0.1471145519580031
fashion 0.15605776370152577
babysitter 0.2448051299671224
engineer -0.24223725681660616
pilot -0.07853385488822553
computer -0.14053746067593784
singer 0.10582590977203045


As we can see from the result above, it turns out that all of the thing that seems unrelated to gender has some biases in them. For example, the word "engineer" has negative value, which means that it leans heavier towards "man", while "babysitter" has positive value, which means that it leans heavier toward "woman". This phenomenon is unacceptable since these words should be neutral and should not reflect any unhealthy gender stereotype.

In order to fix this phenomenon, we need to do some word debiasing such that the words that has no relation whatsoever with gender can be neutralized. Below is the mathematical formulation to neutralize bias in certain word embeddings:


$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g$$
$$e^{debiased} = e - e^{bias\_component}$$

where $e$ is the vector embedding of certain word in a finite dimensional space and $g$ is the bias direction. The formula above takes vector embedding of words and then zeros out the bias direction, creating a new debiased vector embedding representing the same words.

Let's implement formula above in a function.

In [25]:
def word_debiasing(word, vec_gender, word_to_vec_map):
    
    vec_word = word_to_vec_map[word]
    
    vec_word_biased = np.dot(vec_word, vec_gender)*vec_gender/(np.sum(vec_gender**2))
    
    vec_word_debiased = vec_word - vec_word_biased
    
    return vec_word_debiased

Next, we can call the function that we defined above and let's investigate the vector embedding of the word "babysitter" before the application of debiasing and after the application of word debiasing.

In [26]:
e = "babysitter"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["babysitter"], vec_gender))

e_debiased = word_debiasing("babysitter", vec_gender, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, vec_gender))

cosine similarity between babysitter and g, before neutralizing:  0.2448051299671224
cosine similarity between babysitter and g, after neutralizing:  -5.956249596946488e-18


As we can see from the output, before the application of embedding debiasing, the cosine similarity between the word "babysitter" and gender bias direction $g$ is 0.244, which means that this word leans heavier towards the word "woman", "girl", etc. However, after the application of word debiasing, now this word has the cosine similarity of basically 0, which means that this word is no longer associated with a bias towards one gender, which is what we expect.